![Scala Intro](/notebooks/work/misc/thumbnail.jpg)

## Session Objectives

1. Grasp the **fundamentals of FP** by means of Scala
2. Learn the basics of **algebraic data types** (ADTs)
3. Understand the implications of having **functions as first-class citizens**
4. Get used to the **syntax** that simplifies dealing with functions

## So, What is Functional Programming?

Functional Programming is **"Programming With Pure Functions"**

([According to Wikipedia](https://en.wikipedia.org/wiki/Pure_function)) A pure function is a function that has the following properties:
1. Its return value is the same for the same arguments
2. Its evaluation has no side effects

#### Pure function example

In [9]:
def pure(a: Int, b: Int): Int = a + b

defined function pure

In [10]:
pure(1, 2)
pure(1, 2)

res9_0: Int = 3
res9_1: Int = 3

#### Impure function example

In [11]:
var res: Int = 0

def impure(a: Int, b: Int): Int = {
  res += 1
  a + b + res
}

res: Int = 2
defined function impure

In [12]:
impure(1, 2)
impure(1, 2)

res11_0: Int = 4
res11_1: Int = 5

#### Immutability

In [58]:
val x: Int = 3
x = 4

cmd58.sc:2: reassignment to val
val res58_1 = x = 4
                ^Compilation Failed

: 

#### [Why Functional Programming Matters?](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf) (John Hughes)

> *"We conclude that since ***modularity*** is the key to successful programming, functional programming offers important advantages for software development."*

![Indiana Jones](/notebooks/work/misc/grial.jpg)

## Algebraic Data Types

* Make a data type to work with int lists

In [14]:
sealed abstract class IList
class Cons(val head: Int, val tail: IList) extends IList
class End() extends IList

defined class IList
defined class Cons
defined class End

In [23]:
val xs: IList = new Cons(1, new Cons(2, new Cons(3, new End()))) // IList(1, 2, 3)

xs: IList = Cons(1, Cons(2, Cons(3, End())))

* We create a method `prepend` to add a new element in the front of the list

In [24]:
sealed abstract class IList {
  def prepend(h: Int): IList = new Cons(h, this)
}
class Cons(val head: Int, val tail: IList) extends IList
class End() extends IList

defined class IList
defined class Cons
defined class End

In [27]:
val xs: IList = new Cons(1, new Cons(2, new Cons(3, new End()))) // IList(1, 2, 3)
xs.prepend(0)

xs: IList = ammonite.$sess.cmd23$Helper$Cons@590f228b
res26_1: IList = ammonite.$sess.cmd23$Helper$Cons@1da57f91

### Case classes

* They provide common utilities automatically: shorter constructor, `toString` method, `copy` method, etc.

In [45]:
case class Host(name: String, age: Int, show: String)

defined class Host

In [48]:
val jordi = Host("Jordi Hurtado", Int.MaxValue, "Saber y Ganar")

jordi: Host = Host("Jordi Hurtado", 2147483647, "Saber y Ganar")

In [49]:
jordi.copy(show = "Epi y Blas")

res48_0: Host = Host("Jordi Hurtado", 2147483647, "Epi y Blas")
res48_1: Long = 9223372036854775807L

* We can use it to make list creation simpler and to read instance contents

In [37]:
sealed abstract class IList {
  def prepend(h: Int): IList = Cons(h, this)
}
case class Cons(val head: Int, val tail: IList) extends IList
case class End() extends IList

defined class IList
defined class Cons
defined class End

* Creation of list

In [ ]:
val xs = Cons(1, Cons(2, Cons(3, End()))) // IList(1, 2, 3)

### Pattern Matching

* We can use this technique to compare a value against a sequence of patterns

In [32]:
def example(x: Any): Int = x match {
  case i: Int => i
  case s: String => s.length
  case _ => 0
}

example(1)
example("hola")

defined function example
res31_1: Int = 1
res31_2: Int = 4
res31_3: Int = 0

* Pattern matching turns out to be really useful in combination with case classes (`sum` method)

In [36]:
sealed abstract class IList {
  def prepend(h: Int): IList = Cons(h, this)
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
}
case class Cons(val head: Int, val tail: IList) extends IList
case class End() extends IList

defined class IList
defined class Cons
defined class End

* Invocation of `sum` method

In [34]:
val xs: IList = Cons(1, Cons(2, Cons(3, End()))) // IList(1, 2, 3)

xs: IList = Cons(1, Cons(2, Cons(3, End())))

In [35]:
xs.sum

res34: Int = 6

## Lambda Expressions

* So far we've been working with methods

In [50]:
def incr(i: Int): Int = i + 1
incr(0)
incr(33)

defined function incr
res49_1: Int = 1
res49_2: Int = 34

* But functions are *first-class citizens*: ugly lambda expressions

In [28]:
val i: Int = 0
val s: String = "hi"
val incr: Function1[Int, Int] = new Function1[Int, Int] { def apply(i: Int): Int = i + 1 }
incr(0)
incr(33)

i: Int = 0
s: String = "hi"
incr: Int => Int = <function1>
res27_3: Int = 1
res27_4: Int = 34

* We provide a new method, which `maps`s every element in the list

In [57]:
sealed abstract class IList {
  def prepend(h: Int): IList = Cons(h, this)
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
  def map(f: Function1[Int, Int]): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
}
case class Cons(val head: Int, val tail: IList) extends IList
case class End() extends IList

defined class IList
defined class Cons
defined class End

* Invocation of `map` method using `incr`

In [58]:
val incr: Function1[Int, Int] = new Function1[Int, Int] { def apply(i: Int): Int = i + 1 }
val xs: IList = Cons(1, Cons(2, Cons(3, End()))) // IList(1, 2, 3)
val res = xs.map(incr)

incr: Int => Int = <function1>
xs: IList = Cons(1, Cons(2, Cons(3, End())))
res: IList = Cons(2, Cons(3, Cons(4, End())))

## Syntactic Sugar

* An operator is just a method (`concatenate` & `prepend`)

In [64]:
sealed abstract class IList {
  def ::(h: Int): IList = Cons(h, this)
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
  def map(f: Function1[Int, Int]): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}
case class Cons(val head: Int, val tail: IList) extends IList
case class End() extends IList

defined class IList
defined class Cons
defined class End

In [65]:
val xs = Cons(1, Cons(2, End()))
xs.::(0)
0 :: xs

xs: Cons = Cons(1, Cons(2, End()))
res64_1: IList = Cons(0, Cons(1, Cons(2, End())))
res64_2: IList = Cons(0, Cons(1, Cons(2, End())))

In [63]:
val xs = Cons(1, Cons(2, End()))
val ys = Cons(3, End())
xs ++ ys
xs.++(ys)

xs: Cons = Cons(1, Cons(2, End()))
ys: Cons = Cons(3, End())
res62_2: IList = Cons(1, Cons(2, Cons(3, End())))
res62_3: IList = Cons(1, Cons(2, Cons(3, End())))

* It's possible to declare default arguments for parameters

In [69]:
sealed abstract class IList {
  def ::(h: Int): IList = Cons(h, this)
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
  def map(f: Function1[Int, Int]): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}
case class Cons(val head: Int, val tail: IList = End()) extends IList
case class End() extends IList

defined class IList
defined class Cons
defined class End

In [70]:
val xs = Cons(1, Cons(2))

xs: Cons = Cons(1, Cons(2, End()))

* Variadic methods are great for certain situations like creating lists

In [73]:
object IList {
  def crear(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, crear(xs.tail:_*)) 
  }
}

defined object IList

In [74]:
IList.crear(1, 2, 3, 4)

res73: IList = Cons(1, Cons(2, Cons(3, Cons(4, End()))))

* If you use `apply` as a method name, it's applied automatically

In [76]:
object IList {
  def apply(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, apply(xs.tail:_*)) 
  }
}

defined object IList

In [77]:
IList(1, 2, 3, 4)

res76: IList = Cons(1, Cons(2, Cons(3, Cons(4, End()))))

* Scala deploys syntax to simplify the creation of Lambda Expressions 

In [78]:
val incr: Function1[Int, Int] = new Function1[Int, Int] { def apply(i: Int): Int = i + 1 }

incr: Int => Int = <function1>

In [79]:
val incr: Int => Int = (i: Int) => i + 1

incr: Int => Int = ammonite.$sess.cmd78$Helper$$Lambda$3152/2088365499@27ace243

In [80]:
val incr: Int => Int = i => i + 1

incr: Int => Int = ammonite.$sess.cmd79$Helper$$Lambda$3156/1798017984@50c76a02

In [81]:
val incr: Int => Int = _ + 1

incr: Int => Int = ammonite.$sess.cmd80$Helper$$Lambda$3160/817636356@28ee2644

In [85]:
val xs: IList = IList(1, 2, 3, 4, 5)
xs.map(_ * 2)

xs: IList = Cons(1, Cons(2, Cons(3, Cons(4, Cons(5, End())))))
res84_1: IList = Cons(2, Cons(4, Cons(6, Cons(8, Cons(10, End())))))

## Final Result

In [87]:
// IList is an Algebraic Data Type, made of Cons and End
sealed abstract class IList {
    
  // Operators are available
  def ::(h: Int): IList = Cons(h, this)
    
  // Pattern matching is very handy to deal with ADTs
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
    
  // Functions can be passed as parameters
  // `map` is a higher order function
  def map(f: Int => Int): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
    
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}

// Case classes mitigate common boilerplate
// Default parameters could be helpful
case class Cons(val head: Int, val tail: IList = End()) extends IList
case class End() extends IList

object IList {
  // Variadic arguments are idoneous as a list creator
  def apply(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, apply(xs.tail:_*)) 
  }
}

defined class IList
defined class Cons
defined class End
defined object IList

## Takeaways

* Functional programming is programming with pure functions
* Algebraic data types are encoded as a "sum" of case clases
* Functions are treated as first-class citizens. This enables so-called *higher order functions* (HOFs).
* Syntactic sugar is convenient to dulcify expressions

In [88]:
println("Thank You!")

Thank You!


![Applause](/notebooks/work/misc/applause.gif)